In [1]:
import pandas as pd
import plotly.graph_objects as go

from lowe.locations.lookup import name2fips, fips2name, search, generate_df_json
from lowe.acs.ACSClient import ACSClient





In [17]:
cities = ['palm desert, ca']
all_loc = []

for city in cities:
    df = generate_df_json(codetype="city")
    df = df.astype(str)  # Convert all cols to string
    # df[df['A'].str.contains("hello")]
    df = df[df["name"].str.contains(city)]
    fips = df.iloc[0]['fips']

    loc = {
        "state": fips[0:2],
        "city": fips[2:]
    }
    all_loc.append(loc)

In [18]:
all_loc

[{'state': '06', 'city': '55184'}]

In [19]:
fips2name 

<function lowe.locations.lookup.fips2name(loc: Dict[str, str]) -> Dict[str, str]>

In [20]:
client = ACSClient()
await client.initialize()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fdb19898b80>


In [27]:

resp = await client.get_acs(vars = ["S2701"],
                     start_year="2015",
                     end_year="2019",
                     location= all_loc, 
                     estimate="5")

In [28]:
resp1 = await client.get_acs(vars = ["S2701"],
                     start_year="2015",
                     end_year="2019",
                     location= {}, #united states
                     estimate="5")

In [29]:
resp2 = await client.get_acs(vars = ["S2701"],
                     start_year="2015",
                     end_year="2019",
                     location= {'state': '06'}, #california
                     estimate="5")

In [30]:
target_cols = ["SELECTED CHARACTERISTICS OF HEALTH INSURANCE COVERAGE IN THE UNITED STATES Estimate Percent Insured Civilian noninstitutionalized population",
             "location_key"]
  


In [31]:
renamed_colCalifornia = {"SELECTED CHARACTERISTICS OF HEALTH INSURANCE COVERAGE IN THE UNITED STATES Estimate Percent Insured Civilian noninstitutionalized population" : "California"}
renamed_colCity = {"SELECTED CHARACTERISTICS OF HEALTH INSURANCE COVERAGE IN THE UNITED STATES Estimate Percent Insured Civilian noninstitutionalized population" : city.title()}
renamed_colUS = {"SELECTED CHARACTERISTICS OF HEALTH INSURANCE COVERAGE IN THE UNITED STATES Estimate Percent Insured Civilian noninstitutionalized population" : "United States"}


renamed_cols = [renamed_colCalifornia, renamed_colCity, renamed_colUS]
renamed_cols[1]

{'SELECTED CHARACTERISTICS OF HEALTH INSURANCE COVERAGE IN THE UNITED STATES Estimate Percent Insured Civilian noninstitutionalized population': 'Palm Desert, Ca'}

In [16]:
respList = [resp, resp1, resp2]
dictResp = {}
# df_resp= 
# df_resp1 = 
# df_resp2 = 

df_respList= [df_resp, df_resp1, df_resp2]


NameError: name 'resp' is not defined

In [26]:
for i in range(len(respList)):
            df_respList[i] = respList[i][target_cols]
            df_respList[i] = df_respList[i].apply(pd.to_numeric, errors = "ignore")
            df_respList[i] = df_respList[i].rename(columns = renamed_cols[i]) #memorize this

NameError: name 'respList' is not defined

In [32]:
df_resp = resp[target_cols]
df_resp = df_resp.apply(pd.to_numeric, errors = "ignore")
df_resp = df_resp.rename(columns = renamed_colCity) #memorize this
df_resp #city

concept_label,"Palm Desert, Ca",location_key
year,,
2015,89.2,palm desert ca
2016,90.2,palm desert ca
2017,92.4,palm desert ca
2018,92.7,palm desert ca
2019,93.7,palm desert ca


In [33]:
df_resp1 = resp1[target_cols]
df_resp1 = df_resp1.apply(pd.to_numeric, errors = "ignore")
df_resp1 = df_resp1.rename(columns = renamed_colUS) #memorize this
df_resp1

concept_label,United States,location_key
year,,
2015,87.0,us
2016,88.3,us
2017,89.5,us
2018,90.6,us
2019,91.2,us


In [34]:
df_resp2 = resp2[target_cols]
df_resp2 = df_resp2.apply(pd.to_numeric, errors = "ignore")
df_resp2 = df_resp2.rename(columns = renamed_colCalifornia) #memorize this
df_resp2

concept_label,California,location_key
year,,
2015,85.3,ca
2016,87.4,ca
2017,89.5,ca
2018,91.5,ca
2019,92.5,ca


In [35]:
df_final = pd.concat([df_resp[city.title()], df_resp1["United States"]], axis = 1)
df_final = pd.concat([df_final, df_resp2["California"]], axis = 1)
df_final


,"Palm Desert, Ca",United States,California
year,,,
2015,89.2,87.0,85.3
2016,90.2,88.3,87.4
2017,92.4,89.5,89.5
2018,92.7,90.6,91.5
2019,93.7,91.2,92.5


In [39]:
import plotly.express as px

fig = px.line()
#               x=df_final.index, 
#               y=df_final.columns)
#               hover_data={"date": "|%B %d, %Y"},
#               title='custom tick labels with ticklabelmode="period"')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.add_trace(go.Scatter(x = df_final.index, 
                         y = df_final[city.title()],
#                          mode = 'line+markers',
                         name = city.title(),
                         line = dict(color = '#961a30'),
                         legendrank = 3))

fig.add_trace(go.Scatter(x = df_final.index, 
                         y = df_final['California'],
#                          mode = 'line+markers',
                         name = "California",
                         line = dict(color = '#c5485f'),
                         legendrank = 2))

fig.add_trace(go.Scatter(x = df_final.index, 
                         y = df_final['United States'],
#                          mode = 'line+markers',
                         name = "United States",
                         line = dict(color = '#e6aeb7'), 
                         legendrank = 1))

fig.update_traces(texttemplate='%{text:.2s}', textposition='middle center')

fig.update_layout(
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-.15,
    xanchor="center",
    x=.5),
    title = f"Percentage of Population with Health Insurance, {city.title()}, CA, and U.S., 2015-2019", 
    template='plotly_white',
    font=dict(
        family="Old-style", size=14, color="Black"),
    yaxis = dict(
    
    )
    
    )



fig.show()
                                         
                                         
                             